In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.metrics import roc_auc_score

from src.misc_utils import binary_classifier_performance, read_pickle_file
from src.plotting_utils import plot_auroc_with_uncertainty

matplotlibrc_path = "../matplotlibrc"
result_path = "TODO"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
exp_id = "TODO"
multi_run = f"{exp_id}_main_multirun"
num_run = "01"


In [ ]:
cfg = read_pickle_file(
    f"{exp_id}_run_{num_run}_config.pkl", os.path.join(result_path, multi_run)
)
models_dict = read_pickle_file(
    f"{exp_id}_run_{num_run}_models.pkl", os.path.join(result_path, multi_run)
)
results_dict = read_pickle_file(
    f"{exp_id}_run_{num_run}_results.pkl", os.path.join(result_path, multi_run)
)

all_results = []
lag_exp = f"{exp_id}_lag_experiment"
for mod in [15 * i for i in range(11)]:
    all_results.append(
        read_pickle_file(
            f"{exp_id}_lag{mod}_results.pkl", os.path.join(result_path, lag_exp)
        )
    )


In [ ]:
scores_ls, labels_ls, individual_aucs = [], [], []
for key in results_dict:
    tup = results_dict[key]
    labels = tup["ys"]
    scores = tup["scores"]
    scores_ls.append(scores)
    labels_ls.append(labels)
    individual_aucs.append(roc_auc_score(labels, scores))

with plt.rc_context(fname=matplotlibrc_path):
    overall_avg_auc, overall_std_auc = plot_auroc_with_uncertainty(scores_ls, labels_ls)

    plt.plot(
        np.linspace(0.0, 1.0, 50),
        np.linspace(0.0, 1.0, 50),
        color="grey",
        linestyle="dashed",
    )

    plt.legend(loc="lower right", fontsize=15)
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.show()



In [ ]:
num_subjects = 19
num_lag = 11
lag_step = 15

auc = np.zeros((num_lag, num_subjects))
acc = np.zeros((num_lag, num_subjects))

for idx, res_dict in enumerate(all_results):
    frac_pos_ls, roc_auc_ls, avg_prc_ls, f1_score_ls, acc_ls = [], [], [], [], []
    for key in res_dict:
        id_res_dic = res_dict[key]
        labels = id_res_dic["ys"]
        preds = id_res_dic["scores"]
        frac_pos, roc_auc, avg_prc, f1_score, acci = binary_classifier_performance(
            preds, labels, print_and_plot=False
        )
        frac_pos_ls.append(frac_pos)
        roc_auc_ls.append(roc_auc)
        f1_score_ls.append(f1_score)
        acc_ls.append(acci)
    auc[idx] = np.array(roc_auc_ls)
    acc[idx] = np.array(acc_ls)

x = [lag_step * i for i in range(num_lag)]



In [ ]:
with plt.rc_context(fname=matplotlibrc_path):
    large_fontsize = 15
    mid_fontsize = 10
    small_fontsize = 8

    figure_width = 12
    with_to_height_ratio = 5 / 17

    fig = plt.figure(figsize=(figure_width, with_to_height_ratio * figure_width))
    fig.patch.set_facecolor("white")
    fig.subplots_adjust(wspace=1.5, hspace=0.5)

    ax = plt.subplot2grid((1, 12), loc=(0, 4), colspan=4, rowspan=1)
    ax.set_title(r"$\bf{e}$", loc="left", fontsize=large_fontsize, y=1.05, x=-0.15)
    ax.set_title("Individual performance", fontsize=mid_fontsize)
    width = 0.7
    y_pos = np.arange(num_subjects)
    individual_aucs = np.array(individual_aucs)
    args = np.argsort(individual_aucs)
    ax.bar(
        y_pos,
        individual_aucs[args],
        width=width,
        align="center",
        color="C2",
        label="AuROC",
    )
    ax.bar(
        y_pos,
        [0.5] * len(individual_aucs),
        width=width,
        align="center",
        alpha=0.5,
        color="black",
    )

    ax.set_xticks(y_pos)
    ax.set_xticklabels([f"{i+1:02d}" for i in args])
    ax.set_ylim((0.0, 1.0))
    ax.set_yticks([0.0, 0.5, 1.0])
    ax.set_ylabel("AuROC", fontsize=mid_fontsize)
    ax.set_xlabel("Patient number", fontsize=mid_fontsize)

    ax = plt.subplot2grid((1, 12), loc=(0, 0), colspan=4, rowspan=1)
    ax.set_title(r"$\bf{d}$", loc="left", fontsize=large_fontsize, y=1.05, x=-0.15)
    ax.set_title(f"Average AuROC: {overall_avg_auc}", fontsize=mid_fontsize)
    plot_auroc_with_uncertainty(
        scores_ls, labels_ls, color="C2", label="AuROC: ", alpha_uncertainty=0.3
    )
    ax.plot(
        np.linspace(0.0, 1.0, 50),
        np.linspace(0.0, 1.0, 50),
        color="grey",
        linestyle="dashed",
    )
    ax.set_xlim((0.0, 1.0))
    ax.set_ylim((0.0, 1.0))
    ax.set_xticks([0.0, 0.5, 1.0])
    ax.set_yticks([0.0, 0.5, 1.0])

    ax.set_xlabel("False positive rate", fontsize=mid_fontsize)
    ax.set_ylabel("True positive rate", fontsize=mid_fontsize)

    ax = plt.subplot2grid((1, 12), loc=(0, 8), colspan=4, rowspan=1)
    ax.set_title(r"$\bf{f}$", loc="left", fontsize=large_fontsize, y=1.05, x=-0.15)
    ax.set_title(
        "Performance for increasing\nprediction horizon", fontsize=mid_fontsize, y=0.95
    )
    ax.plot(x, np.mean(auc, axis=1), "-s", color="C2")
    ax.fill_between(
        x,
        np.mean(auc, axis=1) - np.std(auc, axis=1),
        np.mean(auc, axis=1) + np.std(auc, axis=1),
        alpha=0.3,
        color="C2",
    )
    ax.set_ylabel("AuROC", fontsize=mid_fontsize)
    ax.set_xlabel("prediction horizon (seconds)", fontsize=mid_fontsize)
    ax.set_ylim((0.4, 1.0))
    ax.set_yticks([0.4, 0.6, 0.8, 1.0])

    fig.tight_layout()
    plt.show()
